In [1]:
import pandas as pd
import praw
import datetime        
from time import sleep
import random
import time



## Building

In [3]:
import praw
import pandas as pd
from tqdm import tqdm

In [4]:
reddit = praw.Reddit(
    client_id='fKSnwxwrgn7DXbSDq8oHnw',         # Your app's client ID
    client_secret='xtz8CGEQjdB4SNbM9eHTp82eP0sIdQ', # Your app's client secret
    user_agent='Copenhagen University Data Collection Course, analyzing comment civility, Oliver',       # A descriptive user agent
    username='OkMinute',    # Your Reddit username
    password='Fodbold7'     # Your Reddit password
)

In [11]:
def get_top_posts(subreddit_list, limit=100, sort_by='top', time_filter='all'):
    """
    Fetch top posts from a list of subreddits and return as a DataFrame.
    
    Parameters:
        subreddit_list (list): List of subreddit names.
        limit (int): Number of posts to fetch per subreddit.
        sort_by (str): Sorting method ('top', 'hot', 'new', 'rising').
        time_filter (str): Time filter for sorting ('all', 'day', 'hour', 'month', 'week', 'year'). Only for 'top'.
    
    Returns:
        pd.DataFrame: DataFrame containing post data.
    """
    
    post_list = []
    
    for subreddit_name in subreddit_list:
        try:
            subreddit = reddit.subreddit(subreddit_name)

            if sort_by == 'top':
                posts = subreddit.top(limit=limit, time_filter=time_filter)
            elif sort_by == 'hot':
                posts = subreddit.hot(limit=limit)
            elif sort_by == 'new':
                posts = subreddit.new(limit=limit)
            elif sort_by == 'rising':
                posts = subreddit.rising(limit=limit)
            else:
                raise ValueError("Invalid sort_by value. Use 'top', 'hot', 'new', or 'rising'.")
            
            for post in posts:
                post_list.append({
                    'subreddit': subreddit_name,
                    'title': post.title,
                    'score': post.score,
                    'id': post.id,
                    'url': post.url,
                    'num_comments': post.num_comments,
                    'created_utc': post.created_utc,
                    'selftext': post.selftext,
                    'author': post.author.name if post.author else None,
                    'upvote_ratio': post.upvote_ratio,
                    'permalink': post.permalink,
                    'domain': post.domain,
                    'flair_text': post.link_flair_text,
                    'is_self': post.is_self,
                    'view_count': post.view_count,
                    'media': post.media
                })
        
        except Exception as e:
            print(f"Error fetching posts from r/{subreddit_name}: {e}. Skipping...")

    return pd.DataFrame(post_list)


In [16]:
def get_post_comments(post):
    """
    Fetch all comments for a given post object.
    
    Parameters:
        post (praw.models.Submission): The Reddit post object.
    
    Returns:
        pd.DataFrame: DataFrame containing comment data.
    """
    post.comments.replace_more(limit=0)  # Remove MoreComments objects
    
    comment_list = []
    for comment in post.comments.list():
        comment_list.append({
            'subreddit': post.subreddit.display_name,
            'post_id': post.id,
            'post_title': post.title,
            'comment_id': comment.id,
            'author': comment.author.name if comment.author else None,
            'score': comment.score,
            'created_utc': comment.created_utc,
            'body': comment.body,
            'parent_id': comment.parent_id
        })
    
    return pd.DataFrame(comment_list)

def get_all_comments(df_posts):
    """
    Loop over posts DataFrame and collect all comments for each post with progress tracking.
    
    Parameters:
        df_posts (pd.DataFrame): DataFrame containing posts.
    
    Returns:
        pd.DataFrame: DataFrame containing all collected comments.
    """
    
    all_comments = []
    for _, post in tqdm(df_posts.iterrows(), total=len(df_posts), desc="Fetching Comments"):
        submission = reddit.submission(id=post['id'])
        comments_df = get_post_comments(submission)
        all_comments.append(comments_df)
        
    

    return pd.concat(all_comments, ignore_index=True) if all_comments else pd.DataFrame()


In [24]:
# Example usage
subreddit_list = [
    'carnivorediet',
    'Carnivore',
    'ketovore',
    'Exvegans',
    'Rawprimal',
    'rawmeat',
    'CarnivoreForum',
    'SaturatedFat',
    'StopEatingSeedOils',
    'Zerocarb',
    'NutritionalPsychiatry',
    'ScientificNutrition',
    'Meatogains',
    'Animalbased',
    'AnimalbasedDiet',
    'Redmeatscience',
    'Carnivorescience',
    'Meatropology',
    'Anti_vegan',
    'AntiVegan',
    'Primaldietcommunity',
    'meatrition',
    'StopEatingFiber',
    'DietitiansSaidWhatNow'
]

df_posts = get_top_posts(subreddit_list, limit=100, sort_by='top', time_filter='year')

# Display DataFrame
df_posts.head()

# Collect all comments for all posts with progress tracking
df_comments = get_all_comments(df_posts)



Fetching Comments: 100%|██████████| 1594/1594 [21:51<00:00,  1.22it/s]  


In [29]:
df_comments.to_csv("Comments.csv", index=False, encoding="utf-8")

In [34]:
df_posts.to_csv("posts.csv")